In [ ]:
!pip install jupyter_dash

In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
import plotly.express as px
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
import itertools

In [ ]:
db = pd.read_csv("https://raw.githubusercontent.com/Hamidreza-Karimian/MLR.csv/main/NewData_OCC50.csv")
db03 = pd.read_csv("https://raw.githubusercontent.com/Hamidreza-Karimian/MLR.csv/main/NewData_OCC51.csv")

In [ ]:
app = JupyterDash()

app.layout = html.Div([
    html.H1(children = 'OCC Algorithm Dashboard ',
           style = {
               'textAlign' : 'center',
               'color' : '#0000FF'
           }
           ),
    html.Div(children = 'Fine-tuning the OCC algorithm parameters and its effect on set-points and SPD',
            style = {
               'textAlign' : 'center',
               'color' : '#0000FF'
            }
            ),
    
    html.Br(),
    html.Br(),
    
    html.Label(children = 'Insert the Learning Rate coefficient please'),
    dcc.Input(
        id='LR_Value',
        placeholder = 'Insert the LR value',
        type = 'number',
        min=0,
        max=1,
        value=0.000001,
        #marks={str(year): str(year) for year in df['year'].unique()},
        step=0.000001
    ),
    
    html.Br(),
    html.Br(),
    
    html.Label(children = 'Choose the Set-Back please'),
    dcc.Dropdown(id="SB_Value",
                 options=[
                     {"label": "18", "value": 18},
                     {"label": "19", "value": 19},
                     {"label": "20", "value": 20},
                     {"label": "21", "value": 21},
                     {"label": "22", "value": 22},
                     {"label": "22.889", "value": 22.889},
                     {"label": "23", "value": 23},
                     {"label": "24", "value": 24},
                     {"label": "25", "value": 25},
                     {"label": "26", "value": 26}],
                 multi=False,
                 value=22.889,
                 style={'width': "50%"}
                 ),
  
    html.Br(),
        
    html.Label(children = 'Choose the Occupant-type please'),
    dcc.Dropdown(id="Occupant_type",
                 options=[
                     {"label": "Tolerant", "value": 1.2},
                     {"label": "Moderate", "value": 2.8},
                     {"label": "Sensitive", "value": 5.4}],
                 multi=False,
                 value="Moderate",
                 style={'width': "50%"}
                 ),
    
    
    dcc.Graph(id='OCC_SP'),
    
    html.Br(),
    html.Br(),

    
    html.Label(children = ' The Set-Point Deviation (SPD) metric was inspired by the calculation of the Heating/Cooling Degree Days (HDD/CDD); In the calculation of the SPD, the monthly deviation of the OCC set-points from a baseline temperature (i.e., 22°C (71.6˚F)) were calculated as a unitless value. positive and higher SPDs indicate better improvement of energy consumption.'),

    html.Br(),
    html.Br(),
    
    html.Label(children = 'The calculated SPD is:', style = {
               'color' : '#ff0000'
            } ),
    
    html.Div(id='SPD'),
    
    html.Br(),
    
    html.Label(children = 'The initial original set-point is:', style = {
               'color' : '#ff0000'
            } ),
    html.Br(),
    html.Label(children = '"23.029"'),
    
    html.Div(id='initial_sp'),
    html.Br(),
    
    
    html.Label(children = 'The resulted final set-point at the end of the period is:', style = {
               'color' : '#ff0000'
            } ),
    
    html.Div(id='final_sp'),
    
    html.Br(),
    html.Br()
    #dcc.Graph(id='SPD')
    
    
])


x = range (0,1)
y= range (18,26,1)
for lr,SB in itertools.zip_longest(x,y):
    
    def lr_coeff(db,lr,SB):
    
        db['OCCSP'] = np.zeros(db.shape[0])
        db['aInc'] = np.zeros(db.shape[0])
        db['bInc'] = np.zeros(db.shape[0])
        db['incP'] = np.zeros(db.shape[0])
        db['SPD'] = np.zeros(db.shape[0])
        db['SPD_Tot'] = np.zeros(db.shape[0])
        db.aInc.iloc[0] = 20
        db.bInc.iloc[0] = -1

        for i in range(db.shape[0]):
            db['incP'].iloc[i] = 1/(1+np.exp(-(db['aInc'][i]+db['bInc'][i]*db.Tin[i])))
            if  19007 < db['Step'][i] <= 42595 :
                db.OCCSP.iloc[i] = (-db['aInc'][i] - 3.029) / db['bInc'][i]
            if  19007 < db['Step'][i] <= 42595 :
                if db['Weekdays'][i] == 'Saturday':
                    db.OCCSP[i] = SB
                if db['Weekdays'][i] == 'Sunday':
                    db.OCCSP[i] = SB
                if db['Time03'][i] < 6:
                    db.OCCSP[i] = SB
                elif db['Time03'][i] > 18:
                    db.OCCSP[i] = SB
            if db['days01'][i] == -89:
                    db.OCCSP[i] =  db['OCCSP'][i-578]

            if i < db.shape[0] :
                if 19007  < db['Step'][i] <= 42595 :
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]            
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db['days01'][i] == -89:
                        db.aInc[i+1] =  db['aInc'][i-578]
                        db.bInc[i+1] =  db['bInc'][i-578]
            db['SPD'] = 22 - db['OCCSP']
            SPD_Tot = (sum(db.SPD)/81)                

        return db

    


@app.callback(
    Output('OCC_SP', 'figure'),
    [Input('LR_Value', 'value'),
    Input('SB_Value', 'value')]
    )

 

def update_figure01(selected_LR,selected_SB):
    filtered_db =  lr_coeff(db, selected_LR, selected_SB)
    
    fig = px.line(filtered_db, x="Time", y=["SP","OCCSP"],
                     #size="pop", color="continent", hover_name="country",
                     #log_x=True, size_max=55
                 )

    return fig
  

#db['SPD'] = np.zeros(db.shape[0])
#db['SPD_Tot'] = np.zeros(db.shape[0])

#def SPD_cal (db, lr,SB): 
#    db['SPD'] = 22 - db['OCCSP']
#    SPD_Tot = (sum(db.SPD)/81)
    
#    return SPD_Tot    


x = range (0,1)
y= range (18,26,1)
for lr,SB in itertools.zip_longest(x,y):
    
    def lr_coeff02(db,lr,SB):
    
        db['OCCSP'] = np.zeros(db.shape[0])
        db['aInc'] = np.zeros(db.shape[0])
        db['bInc'] = np.zeros(db.shape[0])
        db['incP'] = np.zeros(db.shape[0])
        db['SPD'] = np.zeros(db.shape[0])
        db['SPD_Tot'] = np.zeros(db.shape[0])
        db.aInc.iloc[0] = 20
        db.bInc.iloc[0] = -1

        for i in range(db.shape[0]):
            db['incP'].iloc[i] = 1/(1+np.exp(-(db['aInc'][i]+db['bInc'][i]*db.Tin[i])))
            if  19007 < db['Step'][i] <= 42595 :
                db.OCCSP.iloc[i] = (-db['aInc'][i] - 3.029) / db['bInc'][i]
            if  19007 < db['Step'][i] <= 42595 :
                if db['Weekdays'][i] == 'Saturday':
                    db.OCCSP[i] = SB
                if db['Weekdays'][i] == 'Sunday':
                    db.OCCSP[i] = SB
                if db['Time03'][i] < 6:
                    db.OCCSP[i] = SB
                elif db['Time03'][i] > 18:
                    db.OCCSP[i] = SB
            if db['days01'][i] == -89:
                    db.OCCSP[i] =  db['OCCSP'][i-578]

            if i < db.shape[0] :
                if 19007  < db['Step'][i] <= 42595 :
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]            
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db['days01'][i] == -89:
                        db.aInc[i+1] =  db['aInc'][i-578]
                        db.bInc[i+1] =  db['bInc'][i-578]
            db['SPD'] = 22 - db['OCCSP']
            SPD_Tot = (sum(db.SPD)/81)                

        return SPD_Tot

    
@app.callback(
    Output('SPD', 'children'),
    [Input('LR_Value', 'value'),
    Input('SB_Value', 'value')]
    )    
    
    
def update_figure02(selected_LR,selected_SB):
    bar =  lr_coeff02(db, selected_LR, selected_SB)
    
    #SPD_Tot01 = bar ["SPD_Tot"].head(1)
    #fig = px.bar(bar, y=["SPD_Tot"])
                     #size="pop", color="continent", hover_name="country",
                     #log_x=True, size_max=55
        #)

   # fig.update_layout(transition_duration=500)
                   
    #return fig
    return '"{}"'.format(bar)


x = range (0,1)
y= range (18,26,1)
for lr,SB in itertools.zip_longest(x,y):
    
    def lr_coeff03(db,lr,SB):
    
        db['OCCSP'] = np.zeros(db.shape[0])
        db['aInc'] = np.zeros(db.shape[0])
        db['bInc'] = np.zeros(db.shape[0])
        db['incP'] = np.zeros(db.shape[0])
        db['SPD'] = np.zeros(db.shape[0])
        db['SPD_Tot'] = np.zeros(db.shape[0])
        db.aInc.iloc[0] = 20
        db.bInc.iloc[0] = -1

        for i in range(db.shape[0]):
            db['incP'].iloc[i] = 1/(1+np.exp(-(db['aInc'][i]+db['bInc'][i]*db.Tin[i])))
            if  19007 < db['Step'][i] <= 42595 :
                db.OCCSP.iloc[i] = (-db['aInc'][i] - 3.029) / db['bInc'][i]
            if  19007 < db['Step'][i] <= 42595 :
                if db['Weekdays'][i] == 'Saturday':
                    db.OCCSP[i] = SB
                if db['Weekdays'][i] == 'Sunday':
                    db.OCCSP[i] = SB
                if db['Time03'][i] < 6:
                    db.OCCSP[i] = SB
                elif db['Time03'][i] > 18:
                    db.OCCSP[i] = SB
            if db['days01'][i] == -89:
                    db.OCCSP[i] =  db['OCCSP'][i-578]

            if i < db.shape[0] :
                if 19007  < db['Step'][i] <= 42595 :
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db.IncO[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((db03.DcrO01[i]*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db03.IncO04[i] == 0 and db03.DcrO01[i] == 0:
                        db['aInc'][i+1] = db['aInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((0*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]            
                    if db03.IncO04[i] == 1 and db03.DcrO01[i] == 1:
                        db['aInc'][i+1] = db['aInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr
                        db['bInc'][i+1] = db['bInc'][i] + ((1*0.2) - db['incP'][i]) * db['incP'][i] * (1- db['incP'][i]) *lr * db.Tin[i]
                    if db['days01'][i] == -89:
                        db.aInc[i+1] =  db['aInc'][i-578]
                        db.bInc[i+1] =  db['bInc'][i-578]
            db['SPD'] = 22 - db['OCCSP']
            SPD_Tot = (sum(db.SPD)/81)                

        return db['OCCSP'].iloc[23543]

    
@app.callback(
    Output('final_sp', 'children'),
    [Input('LR_Value', 'value'),
    Input('SB_Value', 'value')]
    )    
    
    
def update_figure03(selected_LR,selected_SB):
    Final_SP =  lr_coeff03(db, selected_LR, selected_SB)
    
    #SPD_Tot01 = bar ["SPD_Tot"].head(1)
    #fig = px.bar(bar, y=["SPD_Tot"])
                     #size="pop", color="continent", hover_name="country",
                     #log_x=True, size_max=55
        #)

   # fig.update_layout(transition_duration=500)
                   
    #return fig
    return '"{}"'.format(Final_SP)



In [ ]:
if __name__ == '__main__':
    app.run_server(port = 4050)